In [ ]:
import os
import requests
import numpy as np
import pandas as pd


from google.colab import files

# Optional: upload clustered_output.csv if not already in the environment
uploaded = files.upload()  # choose clustered_output.csv
df = pd.read_csv("clustered_output (1).csv") # Corrected filename

print("Loaded clustered dataset:")
display(df.head())


Saving clustered_output (1).csv to clustered_output (1).csv
Loaded clustered dataset:


,job_title_short,job_country,job_skills,cluster
0,business analyst,afghanistan,"['go', 'oracle']",5
1,business analyst,afghanistan,['sas'],5
2,business analyst,afghanistan,"['go', 'oracle']",5
3,business analyst,afghanistan,"['go', 'oracle']",5
4,business analyst,afghanistan,['sas'],5


In [ ]:
import ast

def ensure_list_skills(sk):
    if isinstance(sk, str):
        try:
            return ast.literal_eval(sk)
        except:
            return []
    elif isinstance(sk, list):
        return sk
    else:
        return []

df['job_skills_list'] = df['job_skills'].apply(ensure_list_skills)
df['cluster'] = df['cluster'].astype(int)



In [ ]:
# ==============================
# Generative AI API configuration
# ==============================
!pip install groq

GAI_API_URL = "https://api.groq.com/openai/v1/chat/completions"
GAI_API_KEY = "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"
MODEL_NAME  = "llama-3.1-8b-instant"










In [ ]:
def call_generative_model(prompt, temperature=0.7, max_tokens=400):
    if not GAI_API_KEY:
        raise ValueError("API key not set. Make sure GAI_API_KEY contains your Groq key.")

    headers = {
        "Authorization": f"Bearer {GAI_API_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a helpful career assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens
    }

    resp = requests.post(GAI_API_URL, headers=headers, json=payload)

    if resp.status_code != 200:
        raise RuntimeError(f"Error from GAI API: {resp.status_code} - {resp.text}")

    data = resp.json()
    return data["choices"][0]["message"]["content"]


In [ ]:
def build_prompt_template_1(job_title, country, suggested_skills, cluster_id, user_goal=None):
    skills_str = ", ".join(suggested_skills) if suggested_skills else "No skills available"

    prompt = f"""
You are a career guidance assistant.

Given the following information:
- Job Title: {job_title}
- Country: {country}
- Cluster ID: {cluster_id}

Suggested important skills for this role:
{skills_str}

Task:
Provide a short explanation (3–4 sentences) about:
1. Why these skills matter for this job.
2. Which 2–3 skills the user should focus on first.

Keep it simple and clear.
"""
    if user_goal:
        prompt += f"\nUser Goal: {user_goal}\n"

    return prompt.strip()


In [ ]:
def build_prompt_template_2(job_title, country, suggested_skills, cluster_id, user_goal=None):
    skills_str = ", ".join(suggested_skills) if suggested_skills else "No skills available"

    prompt = f"""
You are an expert career mentor helping users understand job skill requirements.

User Profile:
- Job Title: {job_title}
- Country: {country}
- Cluster ID: {cluster_id}

System Recommended Skills:
{skills_str}

Task:
1. Briefly describe what this job role generally involves.
2. Group the skills into categories (programming, cloud, data, soft skills, etc.)
3. Suggest a short learning plan for each category.
4. Give 2–3 practical tips for gaining real experience (projects, internships, etc.)
5. Tone should be supportive and motivating.

Use bullet points where helpful.
"""
    if user_goal:
        prompt += f"\nUser Goal: {user_goal}\n"

    return prompt.strip()


In [ ]:
# Pick 3 random rows to avoid rate limit
example_rows = df.sample(n=3, random_state=42)

results = []

for idx, row in example_rows.iterrows():
    job_title = row["job_title_short"]
    country   = row["job_country"]
    skills    = row["job_skills_list"]
    cluster   = int(row["cluster"])

    user_goal = "I want to know what skills to focus on for this job."

    # Build prompts
    prompt1 = build_prompt_template_1(job_title, country, skills, cluster, user_goal)
    prompt2 = build_prompt_template_2(job_title, country, skills, cluster, user_goal)

    # API calls
    output1 = call_generative_model(prompt1)
    output2 = call_generative_model(prompt2)

    results.append({
        "job_title_short": job_title,
        "job_country": country,
        "cluster": cluster,
        "skills_used": skills,
        "template_1_output": output1,
        "template_2_output": output2
    })

df_gai_results = pd.DataFrame(results)
df_gai_results


,job_title_short,job_country,cluster,skills_used,template_1_output,template_2_output
0,software engineer,liberia,4,"[python, aws]","For a software engineer role in Liberia, Pytho...","I'm thrilled to help you, and I'm excited to s..."
1,software engineer,tanzania,4,"[python, go, azure, databricks]","Based on the provided information, here's the ...",I'm excited to help you succeed as a software ...
2,cloud engineer,peru,1,"[python, sql, pandas, flow]","Based on the role of a cloud engineer, here's ...","**Welcome, Cluster 1 User from Peru!**\n\nI'm ..."


# Template Comparison and Analysis

in this phase we have implemented and tested 2 different generative AI prompts , To compare the two prompt templates, we generated model outputs for several randomly sampled job postings.



1.   Personalization and tone:
Template 1: Uses a more neutral, informative tone.

Template 2: Uses a more engaging, motivational tone.


2.   Structure and clarity
Template 1 : Starts with a short contextual introduction then lists  skills, but in a suumrized format.



Template 2 :

Starts with a welcoming intro that anchors the user in their role and context (title, country, cluster), Organizes the response more clearly into sections, and Provides more actionable guidance.


3. Use of Context (Skills, Country, Cluster)

Template 1: Refers to the role and skills only with  high-level advice and the country may be mentioned implicitly

Template 2: More explicitly incorporates context by mentioning country and cluster





# Justification of the Selected Template

Based on the comparison above, we chose Template 2 as the final prompt template for the system.

here is why we selected Template 2:


*  Higher perceived personalization and engagement such as "I'm excited to help you succeed as a .."
*   Clearer structure and more actionable guidance. Template 2 is more structured, with explicit prioritization of skills and concrete steps

*   Better use of contextual details such as country, cluster, skills list
*   Better Alignment With System Goals.






